In [ ]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import os
from __future__ import division
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.model_selection import ParameterGrid
import datetime
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [ ]:
df_train=pd.read_csv('data/atec_anti_fraud_train.csv', parse_dates=['date'])
#df_test=pd.read_csv('data/atec_anti_fraud_test_a.csv', parse_dates=['date'])
df_train_clean=df_train.loc[df_train['label']!=-1]
del df_train

In [ ]:
y=df_train_clean['label']
X=df_train_clean.drop(['id','label','date'],axis=1)
#x_test=df_test.drop(['id','date'],axis=1)
#x_id=df_test['id']

In [ ]:
num_values=2000
category_feature=[]
for i in range(X.shape[1]):
    num = X['f'+str(i+1)].unique().shape[0]
    if num <=num_values:
        category_feature.append(i)

In [ ]:
def atec_metric(preds, train_data):
    labels = train_data.get_label()
    fpr,tpr,threshhold=roc_curve(labels, preds)
    return 'atec_metric', 0.4*tpr[fpr<=0.001][-1] + 0.3*tpr[fpr<=0.005][-1] + 0.3*tpr[fpr<=0.01][-1], True

In [ ]:
feature_name=['feasture_'+str(col+1) for col in range(x_train.shape[1])]
lgb_train=lgb.Dataset(X,y,free_raw_data=False,feature_name=feature_name, categorical_feature=category_feature)

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'device': 'cpu',
    'verbose': 0,
    'early_stopping_round': 100,
    'learning_rate': 0.01,
    'num_leaves': 128,
    'lambda_l1': 0.006,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': [5],
    'bagging_seed':3,
    'feature_fration_seed':2
}

In [ ]:
gbm = lgb.train(params, lgb_train, num_boost_round=10000, valid_sets=lgb_valid, valid_names='valid_set',feval=atec_metric,verbose_eval=False)

In [ ]:
y_pred=gbm.predict(x_test)
data={}
data['id']=x_id
data['prob']=y_pred
df_result=pd.Dataframe(data)
df_result.to_csv('data/prediction.csv')